# Fetch Articles Abstracts from Scopus Search API

### by Miguel Silva Brito

### Dec, 2017

First you need an API KEY from Scopus, get it from https://dev.elsevier.com

Remember to connect to your University VPN first!


In [ ]:
API_KEY=""##### <<<<<<<< CHANGE HERE!!!!

In [1]:
#max number of results to fetch each time depending of Service Level in Scopus API 
COUNT=200

Your query in here

In [2]:
QUERY="MACHINE LEARNING FORECAST DEMAND "

In [3]:
import requests
import json
import pandas as pd

### Search request to Scopus API

In [4]:
def run_scopus_search(start):
    resp = requests.get("http://api.elsevier.com/content/search/scopus?start=%s&count=%s&query=%s" % (str(start),str(COUNT),QUERY),
                        headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    results= json.loads(resp.text.encode('utf-8'))
    return results

### Return scopus IDs from results 

In [5]:
def get_scopus_ids(results):
    return [[str(r['dc:identifier'])] for r in results['search-results']["entry"]]

### Return total numbers of articles found 

In [6]:
def get_totalResults(results):
    return int(results['search-results']['opensearch:totalResults'])

### Return basic info about article

In [7]:
def get_scopus_info(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/abstract/scopus_id/"+ SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})

    return json.loads(resp.text.encode('utf-8'))

### Return abstract article

In [8]:
def get_scopus_text(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/article/scopus_id/" + SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    return json.loads(resp.text.encode('utf-8'))

### Main

run scopus search, return top 200 and number of total articles
loop by next 200 until reach total articles

write dataset to csv file

In [ ]:
start=0
article_data=[]

n=0

results=run_scopus_search(start)
totalResults=get_totalResults(results)

scopus_ids=get_scopus_ids(results)
n_ids_fetched =len(scopus_ids)
print("number of scopus ids fetched : %d de %d" % (n_ids_fetched,totalResults))

while (n<totalResults):
    for scopus_id in scopus_ids:
        results_info = get_scopus_info(scopus_id[0])
        results_text = get_scopus_text(scopus_id[0])
        title=results_info['abstracts-retrieval-response']['coredata']['dc:title']
        pubDate=results_info['abstracts-retrieval-response']['coredata']['prism:coverDate']
        try:
            abstract=str(results_text['full-text-retrieval-response']['coredata']['dc:description'])
            abstract=abstract.replace('Abstract\n               \n                  ','')
        except:
            abstract='(not available)'
        print("fetching %d / %d..." % (n+1,totalResults)) 
        n=n+1
        author=[[str(row['ce:indexed-name'])] for row in results_info['abstracts-retrieval-response']['coredata']['dc:creator']['author']]
        article_data.append({'Scopus_ID' : scopus_id[0],'title':title,'pubDate':pubDate,'abstract': abstract,'author': author[0][0]})
    article_set=pd.DataFrame(article_data)
    article_data=[]
    if (start==0):
        article_set.to_csv("./data/scopus_search_base.csv")
    else:
        article_set.to_csv("./data/scopus_search_base.csv","a")
    start=start+n_ids_fetched
    results=run_scopus_search(start)
    scopus_ids=get_scopus_ids(results)
    n_ids_fetched =len(scopus_ids)
    print("number of scopus ids fetched : %d de %d" % (n_ids_fetched+start,totalResults))
print("Finish!")

number of scopus ids fetched : 200 de 4600
fetching 1 / 4600...
fetching 2 / 4600...
fetching 3 / 4600...
fetching 4 / 4600...
fetching 5 / 4600...
fetching 6 / 4600...
fetching 7 / 4600...
fetching 8 / 4600...
fetching 9 / 4600...
fetching 10 / 4600...
fetching 11 / 4600...
fetching 12 / 4600...
fetching 13 / 4600...
fetching 14 / 4600...
fetching 15 / 4600...
fetching 16 / 4600...
fetching 17 / 4600...
fetching 18 / 4600...
fetching 19 / 4600...
fetching 20 / 4600...
fetching 21 / 4600...
fetching 22 / 4600...
fetching 23 / 4600...
fetching 24 / 4600...
fetching 25 / 4600...
fetching 26 / 4600...
fetching 27 / 4600...
fetching 28 / 4600...
fetching 29 / 4600...
fetching 30 / 4600...
fetching 31 / 4600...
fetching 32 / 4600...
fetching 33 / 4600...
fetching 34 / 4600...
fetching 35 / 4600...
fetching 36 / 4600...
fetching 37 / 4600...
fetching 38 / 4600...
fetching 39 / 4600...
fetching 40 / 4600...
fetching 41 / 4600...
fetching 42 / 4600...
fetching 43 / 4600...
fetching 44 / 4600..

In [ ]:
article_set=pd.read_csv("./data/scopus_search_base.csv") 
print(article_set.info)
print(article_set.head)